In [1]:
## === 0) Notebook header & packages (self-contained) ===============
using Pkg

# Core
using LinearAlgebra, Statistics, Random, ProgressMeter, SparseArrays

# Data & distributions
using DataFrames
using Distributions

println("Julia version: ", VERSION)
println("Number of threads: ", Threads.nthreads())
println("Current dir: ", pwd())

Julia version: 1.10.10
Number of threads: 10
Current dir: /Users/ramzi.chariag/Documents/CEU/PhD/Research/Cov matrix Julia /code


In [2]:
## === 1) Parameters ==================================================
# Complete set of simulation and estimation parameters.
# This replicates the structure of your original `params.jl`,
# except it omits any smoke-test-specific flags or file paths.

PARAMS = (; 
    # =======================
    # --- Panel dimensions ---
    # =======================
    N1 = 5,                       # i dimension (fixed)
    start_N2 = 8,                 # j dimension start
    N2_increment = 4,             # increment of N2 per sample size
    start_T  = 24,                 # t dimension start
    T_increment  = 4,             # increment of T per sample size
    num_sample_sizes = 1,         # number of (N2, T) pairs to generate
    num_reps = 100,                # number of replications per sample size

    # =====================================
    # --- True parameters in the DGP ---
    # =====================================
    beta_true = 2.0,              # true slope coefficient
    c_true    = 0.0,              # true intercept

    # ==============================
    # --- Variance components ---
    # ==============================
    sigma_i = 10.0,                # variance of i effects
    sigma_j = 10.0,                # variance of j effects
    sigma_t = 10.0,                # variance of t effects
    sigma_u = 3.0,                # idiosyncratic variance

    # ==============================
    # --- Mean components ---
    # ==============================
    mu_x = 0.0,                   # mean of regressor x
    sigma_x = 1.0,                # std of regressor x
    mu_u = 0.0,                   # mean of error term u

    # =========================================
    # --- Block-diagonal Ω choices (true) ---
    # =========================================
    i_block = true,               # true => full SPD, false => diagonal
    j_block = false,
    t_block = false,

    # =========================================
    # --- Draw modes for fixed effects ---
    # =========================================
    # Supported: :draw_once | :mixed | :full_redraw
    i_draw_mode = :mixed,
    j_draw_mode = :draw_once,
    t_draw_mode = :draw_once,

    # =========================================
    # --- Optional FE scaling for mixed modes ---
    # =========================================
    E_i = 1.0,
    E_j = 1.0,
    E_t = 1.0,

    # =========================================
    # --- Ω estimation and repetition logic ---
    # =========================================
    i_block_est = true,           # estimate Ω_i as SPD (true) or diagonal (false)
    j_block_est = false,
    t_block_est = false,

    repeat_alpha_gls = true,      # repeat Ω_i along α dimension
    repeat_gamma_gls = false,      # repeat Ω_j along γ dimension
    repeat_lambda_gls = false,     # repeat Ω_t along λ dimension

    repeat_alpha_fgls = true,
    repeat_gamma_fgls = false,
    repeat_lambda_fgls = false,

    repeat_alpha_fgls2 = true,
    repeat_gamma_fgls2 = false,
    repeat_lambda_fgls2 = false,

    # =========================================
    # --- FGLS tuning flags ---
    # =========================================
    subtract_sigma_u2_fgls1 = true,
    subtract_sigma_u2_fgls2 = true,

    # =========================================
    # --- Shrinkage / SPD projection ---
    # =========================================
    gls_shrinkage = 0.0,          # no shrinkage by default
    fgls_shrinkage = 0.0,
    project_spd = true,           # ensure Ω̂ SPD
    spd_floor   = 1e-8,           # minimum eigenvalue floor for SPD projection

    # =========================================
    # --- Output & reproducibility ---
    # =========================================
    seed = 42,                    # master RNG seed
    pin_omega_across_reps = true, # keep same Ω across reps of same size
)


(N1 = 5, start_N2 = 8, N2_increment = 4, start_T = 24, T_increment = 4, num_sample_sizes = 1, num_reps = 100, beta_true = 2.0, c_true = 0.0, sigma_i = 10.0, sigma_j = 10.0, sigma_t = 10.0, sigma_u = 3.0, mu_x = 0.0, sigma_x = 1.0, mu_u = 0.0, i_block = true, j_block = false, t_block = false, i_draw_mode = :mixed, j_draw_mode = :draw_once, t_draw_mode = :draw_once, E_i = 1.0, E_j = 1.0, E_t = 1.0, i_block_est = true, j_block_est = false, t_block_est = false, repeat_alpha_gls = true, repeat_gamma_gls = false, repeat_lambda_gls = false, repeat_alpha_fgls = true, repeat_gamma_fgls = false, repeat_lambda_fgls = false, repeat_alpha_fgls2 = true, repeat_gamma_fgls2 = false, repeat_lambda_fgls2 = false, subtract_sigma_u2_fgls1 = true, subtract_sigma_u2_fgls2 = true, gls_shrinkage = 0.0, fgls_shrinkage = 0.0, project_spd = true, spd_floor = 1.0e-8, seed = 42, pin_omega_across_reps = true)

In [3]:
## === 2) Full DGP ====================================================
# Utility: random SPD with target average diagonal = variance
random_spd(n::Int; variance::Real=1.0, rng::AbstractRNG=Random.GLOBAL_RNG) = begin
    A = randn(rng, n, n)
    Σ = (A * A') / n
    s = variance / mean(diag(Σ))
    Symmetric(Σ .* s)
end

# Utility: homoskedastic diagonal covariance
homoskedastic_diag(n::Int; σ::Real=1.0) = Symmetric(Matrix(I, n, n) .* (σ^2))

# Utility: multivariate normal draw with covariance Σ (zero mean)
mvn_draw(Σ::AbstractMatrix; rng::AbstractRNG) = rand(rng, MvNormal(Σ))

# Build (i,j,t) id vectors with i-fastest ordering (t, j, i loops)
function build_ids(N1::Int, N2::Int, T::Int)
    n = N1 * N2 * T
    i_vec = Vector{Int}(undef, n)
    j_vec = Vector{Int}(undef, n)
    t_vec = Vector{Int}(undef, n)
    idx = 1
    @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
        i_vec[idx] = ii
        j_vec[idx] = jj
        t_vec[idx] = tt
        idx += 1
    end
    return i_vec, j_vec, t_vec
end

# Optional sanity checks for invariance by mode
function assert_drawmode_invariance(df::DataFrame;
        i_draw_mode::Symbol, j_draw_mode::Symbol, t_draw_mode::Symbol)

    constant_within(df, keys::Vector{Symbol}, col::Symbol; atol=1e-10) =
        all([all(isapprox.(g[!, col], first(g[!, col]); atol=atol)) for g in groupby(df, keys)])

    if i_draw_mode == :draw_once
        @assert constant_within(df, [:i], :fe_i) "i:draw_once violated (fe_i not constant across j,t for each i)"
    elseif i_draw_mode == :mixed
        @assert constant_within(df, [:i, :j], :fe_i) "i:mixed violated (fe_i not constant across t for each (i,j))"
    elseif i_draw_mode == :full_redraw
        # no constraint
    else
        error("Unknown i_draw_mode = $i_draw_mode")
    end

    if j_draw_mode == :draw_once
        @assert constant_within(df, [:j], :fe_j) "j:draw_once violated (fe_j not constant across i,t for each j)"
    elseif j_draw_mode == :mixed
        @assert constant_within(df, [:i, :j], :fe_j) "j:mixed violated (fe_j not constant across t for each (i,j))"
    elseif j_draw_mode == :full_redraw
        # no constraint
    else
        error("Unknown j_draw_mode = $j_draw_mode")
    end

    if t_draw_mode == :draw_once
        @assert constant_within(df, [:t], :fe_t) "t:draw_once violated (fe_t not constant across i,j for each t)"
    elseif t_draw_mode == :mixed
        @assert constant_within(df, [:j, :t], :fe_t) "t:mixed violated (fe_t not constant across i for each (j,t))"
    elseif t_draw_mode == :full_redraw
        # no constraint
    else
        error("Unknown t_draw_mode = $t_draw_mode")
    end

    return true
end

# Compose Ω_i, Ω_j, Ω_t from flags and sigmas
function build_cov_mats(N1::Int, N2::Int, T::Int;
        i_block::Bool, j_block::Bool, t_block::Bool,
        sigma_i::Real, sigma_j::Real, sigma_t::Real,
        rng::AbstractRNG)
    Ωi = i_block ? random_spd(N1; variance=sigma_i^2, rng=rng) : homoskedastic_diag(N1; σ=sigma_i)
    Ωj = j_block ? random_spd(N2; variance=sigma_j^2, rng=rng) : homoskedastic_diag(N2; σ=sigma_j)
    Ωt = t_block ? random_spd(T;  variance=sigma_t^2, rng=rng) : homoskedastic_diag(T;  σ=sigma_t)
    return Ωi, Ωj, Ωt
end

"""
generate_dataset(P; [overrides...]) -> (df::DataFrame, meta::NamedTuple)

Creates full panel with columns:
  :i, :j, :t, :x, :u_ijt, :fe_i, :fe_j, :fe_t, :y

Draw modes:
  :draw_once   => FE is a function of that index only (e.g., fe_i[i])
  :mixed       => FE is constant within one “other” dimension group
                   - i : constant over t within each (i,j)
                   - j : constant over t within each (i,j)
                   - t : constant over i within each (j,t)
  :full_redraw => FE varies by observation (i,j,t) with given variance scale E_*

Note: “mixed” uses independent normals at the indicated grouping level.
"""
## === Unified generate_dataset (handles N2/T fallback + fixed Ω) ===
function generate_dataset(P::NamedTuple; kwargs...)
    # Merge inline overrides onto P
    p = merge(P, (; kwargs...))

    # Derive N2/T even if only start_* are present
    N1 = p.N1
    N2 = haskey(p, :N2) ? p.N2 : get(p, :start_N2, nothing)
    T  = haskey(p, :T)  ? p.T  : get(p, :start_T,  nothing)
    @assert N2 !== nothing "Missing N2 (or start_N2) in PARAMS/overrides"
    @assert T  !== nothing "Missing T (or start_T) in PARAMS/overrides"
    N2 = Int(N2); T = Int(T)

    n = N1 * N2 * T
    rng = MersenneTwister(p.seed)

    # Use fixed Ω if provided, else build fresh from flags/sigmas
    Ωi = get(kwargs, :Ωi_fixed, nothing)
    Ωj = get(kwargs, :Ωj_fixed, nothing)
    Ωt = get(kwargs, :Ωt_fixed, nothing)
    if (Ωi === nothing) || (Ωj === nothing) || (Ωt === nothing)
        Ωi, Ωj, Ωt = build_cov_mats(N1, N2, T;
            i_block=p.i_block, j_block=p.j_block, t_block=p.t_block,
            sigma_i=p.sigma_i, sigma_j=p.sigma_j, sigma_t=p.sigma_t,
            rng=rng)
    end

    # Indices
    i_vec, j_vec, t_vec = build_ids(N1, N2, T)

    # --- i fixed effect vector over observations ---
    fe_i_vec = Vector{Float64}(undef, n)
    if p.i_draw_mode == :draw_once
        a_i = mvn_draw(Ωi; rng=rng)                    # length N1
        @inbounds for k in eachindex(i_vec); fe_i_vec[k] = a_i[i_vec[k]]; end
    elseif p.i_draw_mode == :mixed
        a_ij = Dict{Tuple{Int,Int},Float64}()
        @inbounds for jj in 1:N2, ii in 1:N1
            a_ij[(ii,jj)] = rand(rng, Normal(0, p.E_i * p.sigma_i))
        end
        idx = 1
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            fe_i_vec[idx] = a_ij[(ii,jj)]
            idx += 1
        end
    elseif p.i_draw_mode == :full_redraw
        fe_i_vec .= rand(rng, Normal(0, p.E_i * p.sigma_i), n)
    else
        error("Unknown i_draw_mode = $(p.i_draw_mode)")
    end

    # --- j fixed effect vector over observations ---
    fe_j_vec = Vector{Float64}(undef, n)
    if p.j_draw_mode == :draw_once
        a_j = mvn_draw(Ωj; rng=rng)                    # length N2
        @inbounds for k in eachindex(j_vec); fe_j_vec[k] = a_j[j_vec[k]]; end
    elseif p.j_draw_mode == :mixed
        a_ij = Dict{Tuple{Int,Int},Float64}()
        @inbounds for jj in 1:N2, ii in 1:N1
            a_ij[(ii,jj)] = rand(rng, Normal(0, p.E_j * p.sigma_j))
        end
        idx = 1
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            fe_j_vec[idx] = a_ij[(ii,jj)]
            idx += 1
        end
    elseif p.j_draw_mode == :full_redraw
        fe_j_vec .= rand(rng, Normal(0, p.E_j * p.sigma_j), n)
    else
        error("Unknown j_draw_mode = $(p.j_draw_mode)")
    end

    # --- t fixed effect vector over observations ---
    fe_t_vec = Vector{Float64}(undef, n)
    if p.t_draw_mode == :draw_once
        a_t = mvn_draw(Ωt; rng=rng)                    # length T
        @inbounds for k in eachindex(t_vec); fe_t_vec[k] = a_t[t_vec[k]]; end
    elseif p.t_draw_mode == :mixed
        a_jt = Dict{Tuple{Int,Int},Float64}()
        @inbounds for tt in 1:T, jj in 1:N2
            a_jt[(jj,tt)] = rand(rng, Normal(0, p.E_t * p.sigma_t))
        end
        idx = 1
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            fe_t_vec[idx] = a_jt[(jj,tt)]
            idx += 1
        end
    elseif p.t_draw_mode == :full_redraw
        fe_t_vec .= rand(rng, Normal(0, p.E_t * p.sigma_t), n)
    else
        error("Unknown t_draw_mode = $(p.t_draw_mode)")
    end

    # x and u
    x = rand(rng, Normal(p.mu_x, p.sigma_x), n)
    u = rand(rng, Normal(p.mu_u, p.sigma_u), n)

    # Assemble full DataFrame
    df = DataFrame(i=i_vec, j=j_vec, t=t_vec,
                   x=x, u_ijt=u,
                   fe_i=fe_i_vec, fe_j=fe_j_vec, fe_t=fe_t_vec)

    # Optional invariance checks
    assert_drawmode_invariance(df;
        i_draw_mode=p.i_draw_mode, j_draw_mode=p.j_draw_mode, t_draw_mode=p.t_draw_mode)

    # y
    df.y = p.c_true .+ p.beta_true .* df.x .+ df.fe_i .+ df.fe_j .+ df.fe_t .+ df.u_ijt

    meta = (; Ωi=Ωi, Ωj=Ωj, Ωt=Ωt, seed=p.seed, N1=N1, N2=N2, T=T)
    return df, meta
end


# --- Run once with current PARAMS ---
df, meta = generate_dataset(PARAMS)
first(df, 10)


Row,i,j,t,x,u_ijt,fe_i,fe_j,fe_t,y
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,1,0.332267,1.07701,5.69829,-8.94518,-2.0401,-3.54546
2,2,1,1,-1.1501,-0.417333,-14.2206,-8.94518,-2.0401,-27.9234
3,3,1,1,-0.199877,-1.28968,-3.72401,-8.94518,-2.0401,-16.3987
4,4,1,1,-0.693392,1.12948,3.6901,-8.94518,-2.0401,-7.55249
5,5,1,1,0.186594,-1.87735,-0.0761298,-8.94518,-2.0401,-12.5656
6,1,2,1,-0.95745,1.53448,5.62669,-19.9475,-2.0401,-16.7414
7,2,2,1,0.436618,-2.45064,1.06869,-19.9475,-2.0401,-22.4964
8,3,2,1,-2.02603,3.99391,5.69458,-19.9475,-2.0401,-16.3512
9,4,2,1,0.560561,3.23812,6.81085,-19.9475,-2.0401,-10.8176


In [4]:
## === 2A) Patch generate_dataset: allow fixed Ω (pin across reps) ===
# Re-define generate_dataset, adding optional Ωi_fixed/Ωj_fixed/Ωt_fixed kwargs.
# (This overwrites the previous method in this notebook.)
function generate_dataset(P::NamedTuple; kwargs...)
    # Merge inline overrides onto P
    p = merge(P, (; kwargs...))

    N1, N2, T = p.N1, p.N2, p.T
    n = N1 * N2 * T
    rng = MersenneTwister(p.seed)

    # If fixed Ω provided, use them; otherwise build fresh.
    Ωi = get(kwargs, :Ωi_fixed, nothing)
    Ωj = get(kwargs, :Ωj_fixed, nothing)
    Ωt = get(kwargs, :Ωt_fixed, nothing)
    if (Ωi === nothing) || (Ωj === nothing) || (Ωt === nothing)
        Ωi, Ωj, Ωt = build_cov_mats(N1, N2, T;
            i_block=p.i_block, j_block=p.j_block, t_block=p.t_block,
            sigma_i=p.sigma_i, sigma_j=p.sigma_j, sigma_t=p.sigma_t,
            rng=rng)
    end

    # Build indices
    i_vec, j_vec, t_vec = build_ids(N1, N2, T)

    # --- i fixed effect vector over observations ---
    fe_i_vec = similar(Vector{Float64}(undef, n))
    if p.i_draw_mode == :draw_once
        a_i = mvn_draw(Ωi; rng=rng)                    # length N1
        @inbounds for k in eachindex(i_vec); fe_i_vec[k] = a_i[i_vec[k]]; end
    elseif p.i_draw_mode == :mixed
        a_ij = Dict{Tuple{Int,Int},Float64}()
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            key = (ii,jj)
            v = get!(a_ij, key, rand(rng, Normal(0, p.E_i * p.sigma_i)))
        end
        idx = 1
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            fe_i_vec[idx] = a_ij[(ii,jj)]
            idx += 1
        end
    elseif p.i_draw_mode == :full_redraw
        fe_i_vec .= rand(rng, Normal(0, p.E_i * p.sigma_i), n)
    else
        error("Unknown i_draw_mode = $(p.i_draw_mode)")
    end

    # --- j fixed effect vector over observations ---
    fe_j_vec = similar(Vector{Float64}(undef, n))
    if p.j_draw_mode == :draw_once
        a_j = mvn_draw(Ωj; rng=rng)                    # length N2
        @inbounds for k in eachindex(j_vec); fe_j_vec[k] = a_j[j_vec[k]]; end
    elseif p.j_draw_mode == :mixed
        a_ij = Dict{Tuple{Int,Int},Float64}()
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            key = (ii,jj)
            v = get!(a_ij, key, rand(rng, Normal(0, p.E_j * p.sigma_j)))
        end
        idx = 1
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            fe_j_vec[idx] = a_ij[(ii,jj)]
            idx += 1
        end
    elseif p.j_draw_mode == :full_redraw
        fe_j_vec .= rand(rng, Normal(0, p.E_j * p.sigma_j), n)
    else
        error("Unknown j_draw_mode = $(p.j_draw_mode)")
    end

    # --- t fixed effect vector over observations ---
    fe_t_vec = similar(Vector{Float64}(undef, n))
    if p.t_draw_mode == :draw_once
        a_t = mvn_draw(Ωt; rng=rng)                    # length T
        @inbounds for k in eachindex(t_vec); fe_t_vec[k] = a_t[t_vec[k]]; end
    elseif p.t_draw_mode == :mixed
        a_jt = Dict{Tuple{Int,Int},Float64}()
        @inbounds for tt in 1:T, jj in 1:N2
            key = (jj,tt)
            v = get!(a_jt, key, rand(rng, Normal(0, p.E_t * p.sigma_t)))
        end
        idx = 1
        @inbounds for tt in 1:T, jj in 1:N2, ii in 1:N1
            fe_t_vec[idx] = a_jt[(jj,tt)]
            idx += 1
        end
    elseif p.t_draw_mode == :full_redraw
        fe_t_vec .= rand(rng, Normal(0, p.E_t * p.sigma_t), n)
    else
        error("Unknown t_draw_mode = $(p.t_draw_mode)")
    end

    # x and u
    x = rand(rng, Normal(p.mu_x, p.sigma_x), n)
    u = rand(rng, Normal(p.mu_u, p.sigma_u), n)

    # Assemble DataFrame
    df = DataFrame(i=i_vec, j=j_vec, t=t_vec,
                   x=x, u_ijt=u,
                   fe_i=fe_i_vec, fe_j=fe_j_vec, fe_t=fe_t_vec)

    # Optional invariance checks
    assert_drawmode_invariance(df;
        i_draw_mode=p.i_draw_mode, j_draw_mode=p.j_draw_mode, t_draw_mode=p.t_draw_mode)

    # Build y
    y = p.c_true .+ p.beta_true .* df.x .+ df.fe_i .+ df.fe_j .+ df.fe_t .+ df.u_ijt
    df.y = y

    meta = (; Ωi=Ωi, Ωj=Ωj, Ωt=Ωt, seed=p.seed, N1=N1, N2=N2, T=T)
    return df, meta
end

## === 2B) Bundle generator driven 100% by PARAMS ====================

# Size grid from PARAMS (uses N2/T if present, else start_* fields)
function build_size_grid(P::NamedTuple)
    base_N2 = haskey(P, :N2) ? P.N2 : P.start_N2
    base_T  = haskey(P, :T)  ? P.T  : P.start_T
    [(base_N2 + (s-1)*P.N2_increment,
      base_T  + (s-1)*P.T_increment) for s in 1:P.num_sample_sizes]
end

"""
generate_bundle(PARAMS) -> Vector{NamedTuple}

Reads everything from PARAMS:
  - Panel grid: start_N2, N2_increment, start_T, T_increment, num_sample_sizes
  - Repetitions: num_reps
  - Ω choices: i/j/t_block, sigmas, pin_omega_across_reps
  - Reproducibility: seed
Returns a vector over sizes; each entry:
  (N1,N2,T, Ωi,Ωj,Ωt, reps::Vector{NamedTuple})
Where reps[r] = (df, meta, rep).
"""
function generate_bundle(P::NamedTuple)
    @assert haskey(P, :N1) "PARAMS must contain N1"
    @assert all(haskey(P, k) for k in (:start_N2, :N2_increment, :start_T, :T_increment, :num_sample_sizes)) "Missing size grid keys in PARAMS"
    @assert all(haskey(P, k) for k in (:num_reps, :seed)) "Missing repetition/seed keys in PARAMS"

    sizes = build_size_grid(P)
    bundle = Vector{NamedTuple}(undef, length(sizes))

    for (s_idx, (N2, T)) in enumerate(sizes)
        # Draw/Pin Ω for this size (true blocks depend on i_block/j_block/t_block)
        rngΩ = MersenneTwister(hash((P.seed, :OMEGA, s_idx, N2, T)))
        Ωi, Ωj, Ωt = build_cov_mats(P.N1, N2, T;
            i_block=P.i_block, j_block=P.j_block, t_block=P.t_block,
            sigma_i=P.sigma_i, sigma_j=P.sigma_j, sigma_t=P.sigma_t,
            rng=rngΩ)

        reps = Vector{NamedTuple}(undef, P.num_reps)
        for r in 1:P.num_reps
            seed_r = hash((P.seed, :DATA, s_idx, r))
            if P.pin_omega_across_reps
                df, meta = generate_dataset(P; N2=N2, T=T, seed=seed_r,
                    Ωi_fixed=Ωi, Ωj_fixed=Ωj, Ωt_fixed=Ωt)
            else
                df, meta = generate_dataset(P; N2=N2, T=T, seed=seed_r)
            end
            reps[r] = (; df=df, meta=meta, rep=r)
        end

        bundle[s_idx] = (; N1=P.N1, N2=N2, T=T, Ωi=Ωi, Ωj=Ωj, Ωt=Ωt, reps=reps)
    end

    return bundle
end

# --- Usage (no kwargs): ---------------------------------------------
bundle = generate_bundle(PARAMS)

# Peek: list sizes and first 3 rows of the first rep at each size
for (k, b) in enumerate(bundle)
    println("Size $k → N1=$(b.N1), N2=$(b.N2), T=$(b.T), reps=$(length(b.reps))")
    display(first(b.reps[1].df, 10))
end


Row,i,j,t,x,u_ijt,fe_i,fe_j,fe_t,y
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,1,-0.388975,-0.213179,-5.92609,1.59723,-7.80104,-13.121
2,2,1,1,-0.48909,0.554027,-1.32881,1.59723,-7.80104,-7.95677
3,3,1,1,0.961748,-0.780621,-15.5039,1.59723,-7.80104,-20.5648
4,4,1,1,1.46587,3.57077,16.6538,1.59723,-7.80104,16.9526
5,5,1,1,0.000765592,1.47578,-6.72626,1.59723,-7.80104,-11.4527
6,1,2,1,-0.451879,1.84322,-12.3371,11.5689,-7.80104,-7.62976
7,2,2,1,0.28049,5.28915,-2.48844,11.5689,-7.80104,7.12955
8,3,2,1,0.957695,-1.76184,13.5452,11.5689,-7.80104,17.4666
9,4,2,1,-2.23816,-0.564502,3.68073,11.5689,-7.80104,2.40776


Size 1 → N1=5, N2=8, T=24, reps=100


In [5]:
# --- Extract one dataset from bundle --------------------------------
one_df = bundle[1].reps[1].df
one_meta = bundle[1].reps[1].meta
@show size(one_df)

# --- Helper: quick OLS routine -------------------------------------
function ols_estimator(df::DataFrame; x_col=:x, y_col=:y, add_const::Bool=true)
    X = Matrix(df[:, [x_col]])
    if add_const
        X = hcat(ones(size(X,1)), X)
    end
    y = df[:, y_col]
    β = (X'X) \ (X'y)
    e = y - X*β
    σ2 = (e'e) / (length(y) - size(X,2))
    V = σ2 * inv(X'X)
    return β, e, V
end

βols, eols, Vols = ols_estimator(one_df)
println("\nOLS β̂ = ", βols)
println("OLS Var(β̂):\n", Vols)


size(one_df) = (960, 9)

OLS β̂ = [5.393334132887627, 1.218829483992786]
OLS Var(β̂):
[0.350810575420831 0.0037351470789001536; 0.0037351470789001536 0.35037714643968354]


In [6]:
# --- Helper: within transform over arbitrary group vars -------------
function within_transform(df::DataFrame, vars::Vector{Symbol}, col::Symbol)
    # Remove group means sequentially for each variable
    x = copy(df[!, col])
    for v in vars
        gmeans = combine(groupby(df, v), col => mean => :m)
        df_means = leftjoin(df, gmeans, on=v)
        x .-= df_means[:, :m]
    end
    return x
end

# --- OLSFE (demean by i,j,t) ----------------------------------------
function ols_fe(df::DataFrame; x_col=:x, y_col=:y, fe_vars=[:i, :j, :t])
    x_tilde = within_transform(df, fe_vars, x_col)
    y_tilde = within_transform(df, fe_vars, y_col)
    X = hcat(ones(length(x_tilde)), x_tilde)
    β = (X'X) \ (X'y_tilde)
    e = y_tilde - X*β
    σ2 = (e'e) / (length(e) - size(X,2))
    V = σ2 * inv(X'X)
    return β, e, V
end

βfe, efe, Vfe = ols_fe(one_df)
println("\nOLSFE β̂ = ", βfe)
println("OLSFE Var(β̂):\n", Vfe)



OLSFE β̂ = [-10.799633491324327, 1.8269338818749623]
OLSFE Var(β̂):
[0.07989314321173145 -0.0017948974660021693; -0.0017948974660021693 0.08418558078238281]


In [7]:
# --- Helper: Kronecker-style Omega assembly ------------------------
# Builds block-diagonal covariance across i,j,t from Ω_i, Ω_j, Ω_t
function build_big_Omega(Ωi, Ωj, Ωt,
                         N1::Int, N2::Int, T::Int, σu2::Real)
    # Ensure dense matrix form for Kronecker products
    Ωi_m = Matrix(Ωi)
    Ωj_m = Matrix(Ωj)
    Ωt_m = Matrix(Ωt)

    n = N1 * N2 * T
    Ω = kron(Ωt_m, kron(Ωj_m, Ωi_m)) + σu2 * I(n)
    return Symmetric(Ω)
end

# --- Helper: GLS given Omega ---------------------------------------
function gls_estimator(df::DataFrame, Ω::AbstractMatrix; x_col=:x, y_col=:y)
    X = hcat(ones(nrow(df)), Matrix(df[:, [x_col]]))
    y = df[:, y_col]
    # Factor once, then solve
    F = cholesky(Symmetric(Matrix(Ω)); check=false)
    ZX = F \ X
    Zy = F \ y
    XtΩinvX = ZX' * ZX
    XtΩinvy = ZX' * Zy
    β = XtΩinvX \ XtΩinvy
    e = y - X*β
    V = inv(XtΩinvX)          # small (2x2) inverse is fine
    return β, e, V
end


gls_estimator (generic function with 1 method)

In [8]:
############## FGLS1 (single-step) faithful rewrite ##################

# -- diagonal-dominance-safe subtraction (same contract as in your repo)
function diag_dominance_safe_subtract!(A::AbstractMatrix, delta::Real; margin::Real=1e-5)
    @assert size(A,1) == size(A,2)
    n = size(A,1)
    @inbounds for i in 1:n
        s = 0.0
        @inbounds for k in 1:n
            if k != i
                s += abs(A[k,i])
            end
        end
        cap = A[i,i] - (s + margin)
        if cap > 0.0
            A[i,i] -= min(delta, cap)
        end
    end
    return A
end

# -- within-tilde builder for a given suffix
#    suffix → group_vars map as in your code: :i→"alpha", :j→"gamma", :t→"lambda"
function ensure_tilde_columns!(df::DataFrame; x_col::Symbol=:x, y_col::Symbol=:y,
                               suffixes::Vector{String}=String[])
    if isempty(suffixes); return df; end
    rev_map = Dict("alpha"=>:i, "gamma"=>:j, "lambda"=>:t)

    function build_tilde!(df, gv::Vector{Symbol}, suf::String)
        k = length(gv)
        x = df[!, x_col]; y = df[!, y_col]
        xbar_all = mean(x); ybar_all = mean(y)

        xsum = zeros(length(x)); ysum = zeros(length(y))
        for d in gv
            g = combine(groupby(df, d),
                        x_col => mean => :mx,
                        y_col => mean => :my)
            dfm = leftjoin(df, g, on=d)
            xsum .+= dfm.mx
            ysum .+= dfm.my
        end

        xt = x .+ (k-1)*xbar_all .- xsum
        yt = y .+ (k-1)*ybar_all .- ysum

        df[!, Symbol(string(x_col), "_tilde_", suf)] = xt
        df[!, Symbol(string(y_col), "_tilde_", suf)] = yt
        return nothing
    end

    for suf in suffixes
        parts = split(suf, "_")                     # e.g. ["alpha","gamma","lambda"]
        gv = Symbol.(get.(Ref(rev_map), parts, nothing))
        @assert all(.!isnothing.(gv)) "Unknown suffix=$suf"   # <-- FIXED HERE
        build_tilde!(df, gv, suf)
    end
    return df
end


# -- residuals from a tilde suffix
function residuals_from_suffix(df::DataFrame, suffix::AbstractString, beta_hat::Real;
                               x_col::Symbol=:x, y_col::Symbol=:y)
    xt = Symbol(string(x_col), "_tilde_", suffix)
    yt = Symbol(string(y_col), "_tilde_", suffix)
    @assert xt in propertynames(df) && yt in propertynames(df) "Missing tilde columns for suffix=$suffix"
    return df[!, yt] .- beta_hat .* df[!, xt]
end

# -- small-block generator (single-step/raw-moment path)
# component ∈ (:i,:j,:t); suffix chosen per your code:
#  :i → "gamma_lambda", :j → "alpha_lambda", :t → "alpha_gamma"
function generate_single_component_omega(df::DataFrame, component::Symbol,
                                         N1::Int, N2::Int, T::Int,
                                         sigma_u2::Real, beta_hat::Real;
                                         x_col::Symbol=:x, y_col::Symbol=:y)
    suffix = component === :i ? "gamma_lambda" :
             component === :j ? "alpha_lambda" :
             component === :t ? "alpha_gamma" :
             error("component must be :i, :j, or :t")

    ensure_tilde_columns!(df; x_col=x_col, y_col=y_col, suffixes=[suffix])
    res = residuals_from_suffix(df, suffix, beta_hat; x_col=x_col, y_col=y_col)
    @assert length(res) == N1*N2*T

    # Raw-moment OPs (no demeaning in single-step)
    A = reshape(res, N1, N2, T)  # i, j, t
    Ω = if component === :i
        B = reshape(A, N1, N2*T)                       # rows=i, cols=(j,t)
        (B * B') / (N2 * T)
    elseif component === :j
        B = reshape(permutedims(A, (2,1,3)), N2, N1*T) # rows=j, cols=(i,t)
        (B * B') / (N1 * T)
    else # :t
        B = reshape(permutedims(A, (3,2,1)), T, N2*N1) # rows=t, cols=(j,i)
        (B * B') / (N1 * N2)
    end
    return Symmetric(Matrix(Ω))
end

# -- S-matrices (correct shapes under repeat flags)
function make_S_matrices(N1::Int, N2::Int, T::Int;
                         repeat_alpha::Bool,
                         repeat_gamma::Bool,
                         repeat_lambda::Bool)

    n = N1 * N2 * T
    # Sparse identity matrices
    I1 = spdiagm(0 => ones(N1))
    I2 = spdiagm(0 => ones(N2))
    IT = spdiagm(0 => ones(T))

    o1 = sparse(ones(N1,1))
    o2 = sparse(ones(N2,1))
    oT = sparse(ones(T,1))

    # α (i-effects)
    Sα = (repeat_alpha ?
          kron(kron(IT, o2), I1) :                     # n × (T*N1)
          kron(kron(oT, o2), I1))                      # n × N1

    # γ (j-effects)
    Sγ = (repeat_gamma ?
          kron(kron(IT, I2), o1) :                     # n × (T*N2)
          kron(kron(oT, I2), o1))                      # n × N2

    # λ (t-effects)
    Sλ = (repeat_lambda ?
          kron(kron(IT, I2), o1) :                     # n × (T*N2)
          kron(IT, kron(o2, o1)))                      # n × T

    @assert size(Sα,1) == n && size(Sγ,1) == n && size(Sλ,1) == n
    return Sα, Sγ, Sλ
end

# === Construct Ω with sparse S (small blocks dense is fine) ===
construct_omega(Ωa::AbstractMatrix, Ωg::AbstractMatrix, Ωl::AbstractMatrix,
                Sα::AbstractMatrix, Sγ::AbstractMatrix, Sλ::AbstractMatrix,
                sigma_u2::Real) = begin
    n = size(Sα,1)
    # Exploit sparsity in tall S:
    Ω = Sα * Ωa * Sα'
    mul!(Ω, Sγ, Ωg, Sγ', true, 1.0)   # Ω += Sγ*Ωg*Sγ'
    mul!(Ω, Sλ, Ωl, Sλ', true, 1.0)   # Ω += Sλ*Ωl*Sλ'
    Ω .+= sigma_u2 .* I(n)
    Symmetric(Matrix(Ω))
end



# -- Ω assembly with S-matrices (faithful)
construct_omega(Ωa::AbstractMatrix, Ωg::AbstractMatrix, Ωl::AbstractMatrix,
                Sα::AbstractMatrix, Sγ::AbstractMatrix, Sλ::AbstractMatrix,
                sigma_u2::Real) = Symmetric(Sα*Ωa*Sα' .+ Sγ*Ωg*Sγ' .+ Sλ*Ωl*Sλ' .+ sigma_u2.*I(size(Sα,1)))

# -- FGLS1: single-step estimator (faithful)
function fgls1(df::DataFrame, meta::NamedTuple, P::NamedTuple;
               x_col::Symbol=:x, y_col::Symbol=:y)
    N1, N2, T = meta.N1, meta.N2, meta.T

    # 1) OLS slope
    βols, _, _ = ols_estimator(df; x_col=x_col, y_col=y_col)
    β̂ = βols[2]

    # 2) homoskedastic components via tilde residuals
    ensure_tilde_columns!(df; x_col=x_col, y_col=y_col,
                          suffixes=["alpha_gamma_lambda","alpha_gamma","alpha_lambda","gamma_lambda"])
    res_full = residuals_from_suffix(df, "alpha_gamma_lambda", β̂; x_col=x_col, y_col=y_col)
    σu2 = mean(res_full .^ 2)

    res_l = residuals_from_suffix(df, "alpha_gamma", β̂; x_col=x_col, y_col=y_col)
    σλ2 = mean(res_l .^ 2) - σu2

    res_g = residuals_from_suffix(df, "alpha_lambda", β̂; x_col=x_col, y_col=y_col)
    σγ2 = mean(res_g .^ 2) - σu2

    res_a = residuals_from_suffix(df, "gamma_lambda", β̂; x_col=x_col, y_col=y_col)
    σα2 = mean(res_a .^ 2) - σu2

    # 3) base blocks Ω̂ᵢ, Ω̂ⱼ, Ω̂ₜ
    Ωi = (P.i_block_est ?
           generate_single_component_omega(df, :i, N1, N2, T, σu2, β̂;
                                           x_col=x_col, y_col=y_col)
           : Symmetric(Matrix(I(N1) .* max(σα2, 0.0))))

    Ωj = (P.j_block_est ?
           generate_single_component_omega(df, :j, N1, N2, T, σu2, β̂;
                                           x_col=x_col, y_col=y_col)
           : Symmetric(Matrix(I(N2) .* max(σγ2, 0.0))))

    Ωt = (P.t_block_est ?
           generate_single_component_omega(df, :t, N1, N2, T, σu2, β̂;
                                           x_col=x_col, y_col=y_col)
           : Symmetric(Matrix(I(T)  .* max(σλ2, 0.0))))

    # 4) optional σ̂_u²/R subtraction (matching your denominators)
    if P.subtract_sigma_u2_fgls1
        diag_dominance_safe_subtract!(Ωi, σu2 / T)   # i-block: R = T
        diag_dominance_safe_subtract!(Ωj, σu2 / N1)  # j-block: R = N1
        diag_dominance_safe_subtract!(Ωt, σu2 / N1)  # t-block: R = N1 
    end
    Ωi = Symmetric(Matrix(Ωi)); Ωj = Symmetric(Matrix(Ωj)); Ωt = Symmetric(Matrix(Ωt))

    # 5) S matrices + optional repeat along the repeat dimension
    Sα, Sγ, Sλ = make_S_matrices(N1, N2, T;
        repeat_alpha=P.repeat_alpha_fgls, repeat_gamma=P.repeat_gamma_fgls, repeat_lambda=P.repeat_lambda_fgls)

    # if repeat flags are on, blow up the small blocks along the replicate axis
    if P.repeat_alpha_fgls; Ωi = Symmetric(kron(I(T), Matrix(Ωi))); end
    if P.repeat_gamma_fgls; Ωj = Symmetric(kron(I(T), Matrix(Ωj))); end
    if P.repeat_lambda_fgls; Ωt = Symmetric(kron(I(N2), Matrix(Ωt))); end

    # 6) assemble Ω̂
    Ωhat = construct_omega(Ωi, Ωj, Ωt, Sα, Sγ, Sλ, σu2)

    # optional shrinkage and SPD conditioning
    if P.fgls_shrinkage != 1.0
        A = Ωhat isa Symmetric ? parent(Ωhat) : Ωhat
        d = copy(@view A[diagind(A)])
        @. A = P.fgls_shrinkage * A
        @view(A[diagind(A)]) .= d
        Ωhat = Symmetric(A)
    end
    if P.project_spd
        F = eigen(Symmetric(Matrix(Ωhat)))
        d = map(x -> max(x, P.spd_floor), F.values)
        Ωhat = Symmetric(F.vectors * Diagonal(d) * F.vectors')
    end

    # 7) GLS step with Ω̂
    β, e, V = gls_estimator(df, Ωhat; x_col=x_col, y_col=y_col)
    return β, e, V, Ωhat, (; σu2, σα2, σγ2, σλ2)
end

βf1, ef1, Vf1, Ωhat1, sigs1 = fgls1(one_df, one_meta, PARAMS)
println("\nFGLS1 β̂ = ", βf1)
println("FGLS1 Var(β̂):\n", Vf1)


FGLS1 β̂ = [5.228202362912514, 1.304279773612937]
FGLS1 Var(β̂):
[150.89533358240007 1.0047943812690647; 1.0047943812690647 150.90938575103308]


In [9]:
############ FGLS2 (two-step) with pooled σ̂_u² and diag subtraction ############

# --- Safe diagonal subtraction to maintain weak diagonal dominance ---
function diag_dominance_safe_subtract!(A::AbstractMatrix, delta::Real; margin::Real=1e-5)
    @assert size(A,1) == size(A,2)
    n = size(A,1)
    @inbounds for i in 1:n
        s = 0.0
        @inbounds for k in 1:n
            if k != i
                s += abs(A[k,i])
            end
        end
        cap = A[i,i] - (s + margin)
        if cap > 0.0
            di = min(delta, cap)
            A[i,i] -= di
        end
    end
    return A
end

# --- Two-step: average residual vectors over the *repeat* dimension before OPs ---
# resid must be ordered with i-fastest (sort by [:t,:j,:i] before building residuals)
function two_step_mean_outer_products(resid::AbstractVector, component::Symbol,
                                      N1::Int, N2::Int, T::Int)
    @assert length(resid) == N1*N2*T
    A = reshape(resid, N1, N2, T)  # i, j, t (i-fastest)

    if component === :i
        # average over j → X ∈ ℝ^{N1×T}; columns are T replicates
        X = dropdims(mean(A; dims=2), dims=2)
        return Symmetric((X * X') / T)

    elseif component === :j
        # average over i → Y ∈ ℝ^{N2×T}
        Y = dropdims(mean(A; dims=1), dims=1)
        return Symmetric((Y * Y') / T)

    elseif component === :t
        # average over i → Y ∈ ℝ^{N2×T}; then W = Y' ∈ ℝ^{T×N2} (columns are N2 replicates)
        Y = dropdims(mean(A; dims=1), dims=1)
        W = permutedims(Y, (2,1))
        return Symmetric((W * W') / N2)
    else
        error("component must be :i, :j, or :t")
    end
end

# --- Two-step pooled σ̂_u²: average of within-cell sample variances across repeats ---
function two_step_sigma_u(res::AbstractVector{<:Real}, component::Symbol, N1::Int, N2::Int, T::Int)
    @assert length(res) == N1*N2*T
    if component === :i
        # shape: [i, j, t], pool var over t for each (i,j)
        A = reshape(res, N1, N2, T)
        σ2_cells = map(1:N1, 1:N2) do ii, jj
            x = @view A[ii, jj, :]
            T > 1 ? var(x; corrected=true) : 0.0
        end
        return (mean(σ2_cells), T)    # R = T

    elseif component === :j
        # var over i for each (j,t)
        A = reshape(res, N1, N2, T)              # i, j, t
        B = permutedims(A, (2,1,3))              # j, i, t
        σ2_cells = map(1:N2, 1:T) do jj, tt
            x = @view B[jj, :, tt]
            N1 > 1 ? var(x; corrected=true) : 0.0
        end
        return (mean(σ2_cells), N1)  # R = N1

    elseif component === :t
        # var over i for each (j,t) effectively; same R = N1 under current design
        A = reshape(res, N1, N2, T)              # i, j, t
        σ2_cells = map(1:N2, 1:T) do jj, tt
            x = @view A[:, jj, tt]
            N1 > 1 ? var(x; corrected=true) : 0.0
        end
        return (mean(σ2_cells), N1)  # R = N1

    else
        error("component must be :i, :j, or :t")
    end
end

# --- Build residual vector ordered i-fastest ([:t,:j,:i]) using OLS slope ---
function residuals_for_twostep(df::DataFrame, β̂::Real; x_col=:x, y_col=:y)
    dfo = sort(df, [:t, :j, :i])                # i-fastest in memory
    y = dfo[:, y_col]; x = dfo[:, x_col]
    # OLS includes an intercept; build residuals as y - (ĉ + β̂x)
    # We only need β̂ slope here; intercept absorbed in y via mean-structure across groups.
    # If you prefer exact ĉ, pass it too and subtract ĉ below.
    return y .- (β̂ .* x)
end

# --- FGLS2: two-step blocks + pooled σ̂_u² + (σ̂_u²/R) diagonal subtraction ---
function fgls2(df::DataFrame, meta::NamedTuple; x_col=:x, y_col=:y,
               subtract_sigma_u2::Bool=true, shrink_offdiag::Real=1.0,
               project_spd::Bool=false, spd_floor::Real=1e-8)

    N1, N2, T = meta.N1, meta.N2, meta.T

    # 1) OLS slope for residual construction
    βols, _, _ = ols_estimator(df; x_col=x_col, y_col=y_col)
    β̂ = βols[2]

    # 2) Residual vector in i-fastest order
    res = residuals_for_twostep(df, β̂; x_col=x_col, y_col=y_col)

    # 3) Two-step small blocks
    Ωi = two_step_mean_outer_products(res, :i, N1, N2, T)
    Ωj = two_step_mean_outer_products(res, :j, N1, N2, T)
    Ωt = two_step_mean_outer_products(res, :t, N1, N2, T)

    # 4) Two-step pooled σ̂_u² per component + choose pooled value
    σ2_i, Ri = two_step_sigma_u(res, :i, N1, N2, T)
    σ2_j, Rj = two_step_sigma_u(res, :j, N1, N2, T)
    σ2_t, Rt = two_step_sigma_u(res, :t, N1, N2, T)
    # pooled choice: simple mean (you could make this df-weighted if you like)
    σ̂_u2 = mean((σ2_i, σ2_j, σ2_t))

    # 5) Optional subtraction σ̂_u² / R from each block’s diagonal (with dominance cap)
    if subtract_sigma_u2
        diag_dominance_safe_subtract!(Ωi, σ̂_u2 / Ri)
        diag_dominance_safe_subtract!(Ωj, σ̂_u2 / Rj)
        diag_dominance_safe_subtract!(Ωt, σ̂_u2 / Rt)
    end
    Ωi = Symmetric(Matrix(Ωi)); Ωj = Symmetric(Matrix(Ωj)); Ωt = Symmetric(Matrix(Ωt))

    # 6) (Optional) shrink off-diagonals in the big Ω after assembly (kept simple here)
    Ωhat = build_big_Omega(Ωi, Ωj, Ωt, N1, N2, T, σ̂_u2)
    if shrink_offdiag != 1.0
        # simple convex combo with diagonal as a placeholder shrinker:
        D = Diagonal(diag(Ωhat))
        Ωhat = shrink_offdiag * Ωhat .+ (1 - shrink_offdiag) * D
        Ωhat = Symmetric(Matrix(Ωhat))
    end
    if project_spd
        # minimal ridge to enforce SPD
        λmin = eigmin(Symmetric(Matrix(Ωhat)))
        if λmin <= 0
            Ωhat = Symmetric(Matrix(Ωhat) .+ (spd_floor - λmin + 1e-12) .* I)
        end
    end

    # 7) GLS with Ω̂
    β, e, V = gls_estimator(df, Ωhat; x_col=x_col, y_col=y_col)
    return β, e, V, Ωhat, (; σ2_i, σ2_j, σ2_t, σ̂_u2)
end

βfgls2, efgls2, Vfgls2, Ωhat2, sig2_parts = fgls2(one_df, one_meta)
println("\nFGLS2 β̂ = ", βfgls2)
println("FGLS2 Var(β̂):\n", Vfgls2)



FGLS2 β̂ = [0.34817855532513087, 1.7780758351454147]
FGLS2 Var(β̂):
[7342.639834019527 10.781883740119197; 10.781883740119197 17.384035439670953]


In [10]:
## === Oracle GLS (S-matrix assembly, repeat flags, shrink/SPD) ======
function oracle_gls(df::DataFrame, meta::NamedTuple, P::NamedTuple;
                    x_col::Symbol=:x, y_col::Symbol=:y)

    N1, N2, T = meta.N1, meta.N2, meta.T
    σu2 = (haskey(P, :sigma_u2) ? P.sigma_u2 : P.sigma_u^2)

    # True small blocks from the DGP
    Ωi = Symmetric(Matrix(meta.Ωi))
    Ωj = Symmetric(Matrix(meta.Ωj))
    Ωt = Symmetric(Matrix(meta.Ωt))

    # Repeat along the “repeat” dimension (match your FGLS logic)
    if P.repeat_alpha_gls;  Ωi = Symmetric(kron(I(T),  Matrix(Ωi))); end
    if P.repeat_gamma_gls;  Ωj = Symmetric(kron(I(T),  Matrix(Ωj))); end
    if P.repeat_lambda_gls; Ωt = Symmetric(kron(I(N2), Matrix(Ωt))); end

    # S matrices (use GLS repeat flags)
    Sα, Sγ, Sλ = make_S_matrices(N1, N2, T;
        repeat_alpha  = P.repeat_alpha_gls,
        repeat_gamma  = P.repeat_gamma_gls,
        repeat_lambda = P.repeat_lambda_gls)

    # Assemble Ω (true structure + idiosyncratic σu2 I)
    Ωtrue = construct_omega(Ωi, Ωj, Ωt, Sα, Sγ, Sλ, σu2)

    # Optional: shrink off-diagonals and SPD projection
    if P.gls_shrinkage != 1.0
        A = Ωtrue isa Symmetric ? parent(Ωtrue) : Ωtrue
        d = copy(@view A[diagind(A)])
        @. A = P.gls_shrinkage * A             # shrink everything…
        @view(A[diagind(A)]) .= d              # …but keep diagonal fixed
        Ωtrue = Symmetric(A)
    end
    if P.project_spd
        F = eigen(Symmetric(Matrix(Ωtrue)))
        d = map(x -> max(x, P.spd_floor), F.values)
        Ωtrue = Symmetric(F.vectors * Diagonal(d) * F.vectors')
    end

    # GLS
    β, e, V = gls_estimator(df, Ωtrue; x_col=x_col, y_col=y_col)
    return β, e, V, Ωtrue
end

# --- run on your chosen dataset ------------------------------------
β_gls, e_gls, V_gls, Ω_true = oracle_gls(one_df, one_meta, PARAMS)
println("\nOracle GLS β̂ = ", β_gls)
println("Oracle GLS Var(β̂):\n", V_gls)



Oracle GLS β̂ = [5.988895952138123, 1.3563783393664168]
Oracle GLS Var(β̂):
[92.81643962346892 1.4462362075501098; 1.4462362075501098 91.84279024701333]


In [11]:
## === 4) One-shot wrapper to run all estimators on (df, meta) =======
function estimate_all(df::DataFrame, meta::NamedTuple, P::NamedTuple)
    # OLS
    βols, eols, Vols = ols_estimator(df)
    # OLS with i-j-t FE
    βfe, efe, Vfe    = ols_fe(df)

    # FGLS1 (faithful rewrite you have above)
    βf1, ef1, Vf1, Ωhat1, sigs1 = fgls1(df, meta, P)

    # FGLS2 (two-step version from earlier message)
    βf2, ef2, Vf2, Ωhat2, sig2_parts = fgls2(df, meta)

    # Oracle GLS (S-matrix assembly with repeat flags)
    βgls, egls, Vgls, Ωtrue = oracle_gls(df, meta, P)

    return (; 
        β = (; ols=βols[2], fe=βfe[2], fgls1=βf1[2], fgls2=βf2[2], gls=βgls[2]),
        V = (; ols=Vols, fe=Vfe, fgls1=Vf1, fgls2=Vf2, gls=Vgls),
        resid = (; ols=eols, fe=efe, fgls1=ef1, fgls2=ef2, gls=egls),
        omegas = (; fgls1=Ωhat1, fgls2=Ωhat2, gls=Ωtrue),
        sigma_parts = (; fgls1=sigs1, fgls2=sig2_parts),
        meta = meta
    )
end

## === MC summarizer: bias, variance, variance ratio =================
# For each estimator, we compute:
#   bias        = mean(β̂) - β_true
#   var_emp     = var(β̂ across reps)                # empirical sampling variance
#   var_hat_bar = mean( V̂[2,2] across reps )        # average model-based variance
#   var_ratio   = var_hat_bar / var_emp
function summarize_reps(rep_out::Vector{NamedTuple}, β_true::Real)
    ests = [:ols, :fe, :fgls1, :fgls2, :gls]

    # collect β̂ and V̂[2,2]
    β̂ = Dict(name => Float64[] for name in ests)
    v̂ = Dict(name => Float64[] for name in ests)

    for R in rep_out
        push!(β̂[:ols],   R.β.ols);   push!(v̂[:ols],   R.V.ols[2,2])
        push!(β̂[:fe],    R.β.fe);    push!(v̂[:fe],    R.V.fe[2,2])
        push!(β̂[:fgls1], R.β.fgls1); push!(v̂[:fgls1], R.V.fgls1[2,2])
        push!(β̂[:fgls2], R.β.fgls2); push!(v̂[:fgls2], R.V.fgls2[2,2])
        push!(β̂[:gls],   R.β.gls);   push!(v̂[:gls],   R.V.gls[2,2])
    end

    rows = NamedTuple[]
    for name in ests
        mβ   = mean(β̂[name])
        bias = mβ - β_true
        var_emp = var(β̂[name]; corrected=true)
        var_hat_bar = mean(v̂[name])
        var_ratio   = var_hat_bar / var_emp
        push!(rows, (; estimator=String(name),
                     mean_beta=mβ, bias=bias,
                     var_emp=var_emp, var_hat_bar=var_hat_bar,
                     var_ratio=var_ratio))
    end
    return DataFrame(rows)
end

## === 5) Monte Carlo over bundle: outer seq, inner parallel + PB ===
function run_mc_bundle(bundle::Vector{<:NamedTuple}, P::NamedTuple)
    n_sizes = length(bundle)
    results_by_size = Vector{NamedTuple}(undef, n_sizes)

    for s in 1:n_sizes
        b = bundle[s]
        N1, N2, T = b.N1, b.N2, b.T
        R = length(b.reps)

        rep_out = Vector{NamedTuple}(undef, R)

        # Progress bar for the inner (parallel) loop
        desc = "Size $s/$n_sizes  (N1=$N1, N2=$N2, T=$T) → reps"
        p = Progress(R; desc=desc, dt=0.2)

        Threads.@threads for r in 1:R
            df   = b.reps[r].df
            meta = b.reps[r].meta
            rep_out[r] = estimate_all(df, meta, P)
            next!(p)  # thread-safe update
        end
        finish!(p)

        # Summaries (bias, variance, variance ratio)
        summ = summarize_reps(rep_out, P.beta_true)

        results_by_size[s] = (;
            N1=N1, N2=N2, T=T, reps=R,
            summary=summ,
            per_rep=rep_out
        )

        println("Done size $s / $n_sizes → (N1,N2,T)=($N1,$N2,$T); R=$R")
        display(summ)
    end

    return results_by_size
end

run_mc_bundle (generic function with 1 method)

In [12]:
mc_results = run_mc_bundle(bundle, PARAMS)

Size 1/1  (N1=5, N2=8, T=24) → reps 100%|████████████████| Time: 0:04:23


Row,estimator,mean_beta,bias,var_emp,var_hat_bar,var_ratio
,String,Float64,Float64,Float64,Float64,Float64
1,ols,1.95015,-0.0498537,0.262657,0.302037,1.14993
2,fe,1.95274,-0.0472561,0.0915674,0.0907717,0.99131
3,fgls1,1.95573,-0.044265,0.280547,120.243,428.602
4,fgls2,2.03102,0.0310248,2.1308,213.522,100.207
5,gls,1.95943,-0.040574,0.281277,93.0558,330.833


Done size 1 / 1 → (N1,N2,T)=(5,8,24); R=100


1-element Vector{NamedTuple}:
 (N1 = 5, N2 = 8, T = 24, reps = 100, summary = 5×6 DataFrame
 Row │ estimator  mean_beta  bias        var_emp    var_hat_bar  var_ratio 
     │ String     Float64    Float64     Float64    Float64      Float64   
─────┼─────────────────────────────────────────────────────────────────────
   1 │ ols          1.95015  -0.0498537  0.262657     0.302037     1.14993
   2 │ fe           1.95274  -0.0472561  0.0915674    0.0907717    0.99131
   3 │ fgls1        1.95573  -0.044265   0.280547   120.243      428.602
   4 │ fgls2        2.03102   0.0310248  2.1308     213.522      100.207
   5 │ gls          1.95943  -0.040574   0.281277    93.0558     330.833, per_rep = NamedTuple[(β = (ols = 1.218829483992786, fe = 1.8269338818749623, fgls1 = 1.304279773612937, fgls2 = 1.7780758351454147, gls = 1.3563783393664168), V = (ols = [0.350810575420831 0.0037351470789001536; 0.0037351470789001536 0.35037714643968354], fe = [0.07989314321173145 -0.0017948974660021693; -0.0